In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import os
import pickle

In [2]:
from IPython.display import Audio, display

In [3]:
from utils_projet import get_maxima, search_song

# Part 2: Audio fingerprints

## Exercice 4

In [4]:
def compute_spectrogram(x):
    """
    Calcule le spectrogramme du signal x
    Args:
        x (np.array): Le signal audio
    
    Returns:
        S: (np.array): Le spectrogramme
    """

    D = librosa.stft(x , n_fft=2048, hop_length=512, win_length=1024)
    S = np.abs(D)**2

    return S

In [5]:
filename = "datasets/songs_part_2/Classicals.de - Bizet, Georges - Habanera (From the Opera Carmen) - Arranged for Music Box.mp3"

In [6]:
song, fs = librosa.load(filename, sr=3000)

In [7]:
S = compute_spectrogram(song)
print(f"Spectrogramme caclculé, dimensions de la matrice : {S.shape}")

Spectrogramme caclculé, dimensions de la matrice : (1025, 601)


In [8]:
maxima = get_maxima(S)
print(f"Nombre de maxima trouvés : {len(maxima)}")
if(len(maxima) > 0):
    print(f"Coordonnées du premier maximum :\nFreq = {maxima[0, 0]}\nTemps = {maxima[0, 1]}")

Nombre de maxima trouvés : 592
Coordonnées du premier maximum :
Freq = 101
Temps = 55


In [9]:
def get_maxima_in_tz(S, maxima, anchor):
    """
    Retourne les 10 maxima les plus forts situés dans la zone cible de l'ancre
    Version 2, optimisée avec vectorisation NumPy
    
    Args:
        S (np.array): Le spectrogramme complet
        maxima (np.array): Matrice (N, 2) contenant tous les maxima [freq, temps]
        anchor (tuple): Coordonnées de l'ancre [freq_anchor, time_anchor]
        
    Returns:
        np.array: Liste des coordoonées [freq, temps] des 10 meilleurs points
    """
    f_anchor, t_anchor = anchor
    
    t_min = t_anchor + 1
    t_max = t_anchor + 300
    f_min = f_anchor - 10
    f_max = f_anchor + 10
    
    #On crée un masque booléen qui vaut True si le point est dans le rectangle
    mask = (
        (maxima[:, 1] >= t_min) & 
        (maxima[:, 1] <= t_max) & 
        (maxima[:, 0] >= f_min) & 
        (maxima[:, 0] <= f_max)
    )
    
    candidates = maxima[mask]
    
    if len(candidates) == 0:
        return []

    c_freqs = candidates[:, 0].astype(int)
    c_times = candidates[:, 1].astype(int)
    
    amplitudes = S[c_freqs, c_times]
    
    #On garde les 10 plus forts
    top_indices = np.argsort(amplitudes)[-10:][::-1]
    
    return candidates[top_indices]

In [10]:
# On teste avec un point d'ancre fictif (le premier maximum trouvé)
if len(maxima) > 0:
    anchor_point = maxima[0]
    
    points_zone = get_maxima_in_tz(S, maxima, anchor_point)
    
    print(f"Ancre : {anchor_point}")
    print(f"Points trouvés dans la zone cible : {len(points_zone)}")
    print(points_zone)

Ancre : [101  55]
Points trouvés dans la zone cible : 10
[[101 185]
 [101 165]
 [101 145]
 [101 125]
 [101 346]
 [101 306]
 [101  75]
 [101 296]
 [101  65]
 [101 195]]


In [11]:
def get_hashes(anchor, maxima_in_tz):
    """
    Génère des hashes pour les paires (anchor, point_in_tz)
    
    Args:
        anchor (tuple/array): Coordonnées de l'ancre [freq, time]
        maxima_in_tz (list/array): Liste des points [freq, time] dans la zone cible
        
    Returns:
        list: Liste de tuples (hash_value, time_anchor)
    """
    hashes = []
    
    f_anchor = int(anchor[0])
    t_anchor = int(anchor[1])
    
    for point in maxima_in_tz:
        f_point = int(point[0])
        t_point = int(point[1])
        
        delta_t = t_point - t_anchor
        
        hash_val = (f_anchor * 1_000_000) + (f_point * 1_000) + delta_t
        
        hashes.append((hash_val, t_anchor))
        
    return hashes

In [12]:

test_anchor = [100, 50]
test_points_tz = [[105, 60], [95, 80]] 

resultats = get_hashes(test_anchor, test_points_tz)

print("Résultats des Hashes :")
for h, t in resultats:
    print(f"Hash: {h}, Temps Ancrage: {t}")

Résultats des Hashes :
Hash: 100105010, Temps Ancrage: 50
Hash: 100095030, Temps Ancrage: 50


In [13]:
def process_signal(x):
    """
    Traite le signal audio pour en extraire une empreinte
    
    Args:
        x (np.array): Le signal audio brut
        
    Returns:
        dict: Un dictionnaire {hash: temps_ancre}
    """
    S = compute_spectrogram(x)
    
    maxima = get_maxima(S)
    
    song_hashes = {}
    
    for anchor in maxima:
        maxima_tz = get_maxima_in_tz(S, maxima, anchor)
        
        hashes_list = get_hashes(anchor, maxima_tz)
        
        for h, t_anchor in hashes_list:
            song_hashes[h] = int(t_anchor)
            
    return song_hashes

In [14]:
x, fs = librosa.load(filename, sr=3000) # On prend 30s pour aller vite

signature = process_signal(x)

print(f"Nombre de hashs générés : {len(signature)}")
    
print("Exemple de signature (Hash : Temps) :")
for k in list(signature.keys())[:5]:
    print(f"{k} : {signature[k]}")



Nombre de hashs générés : 2790
Exemple de signature (Hash : Temps) :
101101130 : 306
101101110 : 326
101101090 : 346
101101070 : 366
101101291 : 145


## Exercice 5

In [15]:
def create_database(folder_path):
    """
    Parcourt un dossier, calcule les empreintes de tous les fichiers audio
    et sauvegarde le résultat dans un fichier pickle
    
    Args:
        folder_path (str): Le chemin vers le dossier contenant les fichiers audio
        
    Returns:
        list: La liste des dictionnaires de hashs
        list: La liste des noms de fichiers correspondants
    """
    database = []
    song_names = []
    
    valid_extensions = (".mp3", ".wav", ".m4a")
    files = [f for f in os.listdir(folder_path) if f.lower().endswith(valid_extensions)]
    
    if not files:
        print(f"Aucun fichier audio trouvé dans '{folder_path}'")
        return [], []
    
    for i, filename in enumerate(files):
        file_path = os.path.join(folder_path, filename)
                    
        x, fs = librosa.load(file_path, sr=3000)
            
        hashes = process_signal(x)
            
        database.append(hashes)
        song_names.append(filename)
            
    with open("out_exercice4_5/dataset.pickle", "wb") as handle:
        pickle.dump(database, handle)
    print("Base de données sauvegardée dans 'out_exercice4_5/dataset.pickle'.")
    
    with open("out_exercice4_5/song_names.pickle", "wb") as handle:
        pickle.dump(song_names, handle)
        
    return database, song_names

In [16]:
src_path = "datasets/songs_part_2"
out_folder = "out_exercice4_5/"

In [18]:
database_hashes, song_names = create_database(src_path)

print("--- RÉSULTAT DU TEST ---")
print(f"Nombre de chansons traitées : {len(database_hashes)}")
print(f"Noms des fichiers : {song_names}")

Base de données sauvegardée dans 'out_exercice4_5/dataset.pickle'.
--- RÉSULTAT DU TEST ---
Nombre de chansons traitées : 13
Noms des fichiers : ['Classicals.de - Bizet, Georges - Habanera (From the Opera Carmen).mp3', "Classicals.de - Debussy - Children's Corner - VI. Golliwogg's Cakewalk.mp3", 'Classicals.de - Debussy - Suite Bergamasque - 4. Passepied - L.75.mp3', 'Classicals.de - Debussy - Suite Bergamasque - 1. Prelude - L.75.mp3', "Classicals.de - Debussy - Children's Corner - I. Doctor Gradus ad Parnassum.mp3", "Classicals.de - Debussy - Children's Corner - IV. The Snow is Dancing.mp3", 'Classicals.de - Bizet, Georges - Carmen Prelude - Arranged for Solo Piano.mp3', 'Classicals.de - Debussy - Clair de Lune - L. 75 - Concert Grand Version.mp3', 'Classicals.de - Bizet, Georges - Habanera (From the Opera Carmen) - Arranged for Music Box.mp3', "Classicals.de - Debussy - Children's Corner - III. Serenade of the Doll.mp3", "Classicals.de - Debussy - Children's Corner - V. The Little S

In [19]:
dataset_path = out_folder + "dataset.pickle"

In [20]:
def load_database(folder_path):
    """Charge la base de données et les noms depuis les fichiers pickle"""

    with open(folder_path + "dataset.pickle", "rb") as handle:
        db_hashes = pickle.load(handle)
    with open(folder_path + "song_names.pickle", "rb") as handle:
        song_names = pickle.load(handle)
    print(f"Base chargée : {len(db_hashes)} chansons")
    return db_hashes, song_names

def identify_song(excerpt, db_hashes, song_names):
    """
    Identifie un extrait audio en le comparant à la base
    
    Args:
        excerpt (np.array): Le signal audio de l'extrait (5 à 15s)
        db_hashes (list): La base de données des signatures.
        song_names (list): Les noms des chansons
        
    Returns:
        list: Les noms des meilleures correspondances.
    """
    
    print("Calcul de l'empreinte de l'extrait")
    excerpt_hashes = process_signal(excerpt)
    
    if not excerpt_hashes:
        print("Aucune signature trouvée dans l'extrait")
        return []

    top_indices = search_song(db_hashes, excerpt_hashes)
    
    top_matches = [song_names[idx] for idx in top_indices]
    
    return top_matches

In [21]:
db_hashes, song_names = load_database(out_folder)

Base chargée : 13 chansons


In [86]:
test_file_name = song_names[8] 
    
path_to_full_song = os.path.join("datasets/songs_part_2/", test_file_name) 
    
print(f"\nChanson originale : {test_file_name}")
    
full_song, fs = librosa.load(path_to_full_song, sr=3000)


Chanson originale : Classicals.de - Bizet, Georges - Habanera (From the Opera Carmen) - Arranged for Music Box.mp3


In [87]:
start_sec = 10
duration = 10
excerpt = full_song[start_sec*fs : (start_sec+duration)*fs]
    
print(f"Extrait généré : {duration} secondes (à partir de {start_sec}s)")
display(Audio(data=excerpt, rate=3000))

Extrait généré : 10 secondes (à partir de 10s)


In [83]:
#On ajoute du bruit
noise = np.random.randn(len(excerpt))
display(Audio(data=noise, rate=3000))

In [104]:
alpha = 0.02

In [105]:
noisy_version = excerpt + alpha * noise
display(Audio(data=noisy_version, rate=3000))

Test avec la version "pure"

In [89]:
resultats = identify_song(excerpt, db_hashes, song_names)

Calcul de l'empreinte de l'extrait


In [90]:
print("\nRÉSULTATS :")
for i, nom in enumerate(resultats):
    print(f"{i+1}. {nom}")
        
if resultats and resultats[0] == test_file_name:
    print("\nSUCCÈS : La chanson a été correctement identifiée")
else:
    print("\nÉCHEC : La bonne chanson n'est pas en première position")


RÉSULTATS :
1. Classicals.de - Bizet, Georges - Habanera (From the Opera Carmen) - Arranged for Music Box.mp3
2. Classicals.de - Debussy - Suite Bergamasque - 1. Prelude - L.75.mp3
3. Classicals.de - Debussy - Children's Corner - VI. Golliwogg's Cakewalk.mp3

SUCCÈS : La chanson a été correctement identifiée


Test avec la version bruité (2% de bruit)

In [106]:
resultats = identify_song(noisy_version, db_hashes, song_names)

Calcul de l'empreinte de l'extrait


In [107]:
print("\nRÉSULTATS :")
for i, nom in enumerate(resultats):
    print(f"{i+1}. {nom}")
        
if resultats and resultats[0] == test_file_name:
    print("\nSUCCÈS : La chanson a été correctement identifiée")
else:
    print("\nÉCHEC : La bonne chanson n'est pas en première position")


RÉSULTATS :
1. Classicals.de - Bizet, Georges - Habanera (From the Opera Carmen) - Arranged for Music Box.mp3
2. Classicals.de - Debussy - Clair de Lune - L. 75 - Concert Grand Version.mp3
3. Classicals.de - Debussy - Suite Bergamasque - 2. Menuet - L.75.mp3

SUCCÈS : La chanson a été correctement identifiée
